### Image classification results on the chest x-ray dataset

We used the ResNet-50 model and first trained the network with 224×224 input image size. We used data augmentation techniques such as random cropping, and image transformations. Even though a chest x-ray image is different from ImageNet images, using a pre-trained model trained on the ImageNet dataset helps in achieving better classification accuracy. Hence, we used the use_pretrained_model hyperparameter in the Amazon SageMaker image classification algorithm to train the network. Since this is a multi-label classification, we set the multi_label parameter to 1. We resized the chest x-ray images to 256 before training so that the network can crop 224×224 regions from the input image.

The following code snippet shows how it can be done using the [Amazon SageMaker Estimator interface](https://sagemaker.readthedocs.io/en/latest/estimators.html) and the image classification algorithm.

In [ ]:
bucket = 'analytics-serverless-west'
prefix = 'sagemaker/x-ray'

s3train = 's3://{}/{}/train/'.format(bucket, prefix)
print(s3train)

training_image = get_image_uri(sess.boto_region_name, 'image-classification', repo_version="latest")
s3train = 's3://{}/{}/train/'.format(bucket, prefix)
s3validation = 's3://{}/{}/validation/'.format(bucket, prefix)
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

In [ ]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role,
                        train_instance_count=1,
                        train_instance_type='ml.p3.16xlarge',
                        train_volume_size = 50, train_max_run = 360000,
                        input_mode= 'File', output_path=s3_output_location,
                        sagemaker_session=sess)
multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
                                        image_shape = "3,224,224", num_classes=14,
                                        mini_batch_size=256, 
                                        resize=256,  epochs=100, 
                                        learning_rate=0.0005, optimizer='adam', 
                                        num_training_samples=80000,
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float32', multi_label = 1)
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)

### Training with weighted loss

An additional feature introduced in image classification is the use of weighted loss to handle class imbalance. Typically, when training with a multi-label dataset, there might be imbalance between classes. This imbalance can lead to a network leaning towards learning one class over another. To avoid that, the Amazon SageMaker image classification algorithm uses the use_weighted_loss hyperparameter to balance the samples. When this parameter is set to 1, a weight value is calculated for each label based on the number of samples of that label in the training set. First, the number of samples in each class is calculated from the training set and the weight for loss update is set to N/N_l for that class where N is the total number of samples in the training set and N_l is the total number of samples for class l in the training set. This will weigh the loss calculated for gradient update differently for each class based on their weight thereby enabling balanced training. The average AUC increased to 0.814 when trained using the weighted loss feature enabled while still using 224×224 input resolution.

In [ ]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role,
                        train_instance_count=1,
                        train_instance_type='ml.p3.16xlarge',
                        train_volume_size = 50, train_max_run = 360000,
                        input_mode= 'File', output_path=s3_output_location,
                        sagemaker_session=sess)
# multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
#                                         image_shape = "3,224,224", num_classes=14,
#                                         mini_batch_size=256, 
#                                         resize=256,  epochs=100, 
#                                         learning_rate=0.0005, optimizer='adam', 
#                                         num_training_samples=80000,
#                                         augmentation_type = 'crop_color_transform',
#                                         precision_dtype='float32', multi_label = 1)
multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
                                        image_shape = "3,224,224", num_classes=14,
                                        mini_batch_size=256, resize=256,  epochs=100, 
                                        learning_rate=0.0005, optimizer='adam', 
                                        num_training_samples=80000, use_weighted_loss=1,
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float32', multi_label = 1)
train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)

### Training with mixed-precision

The Amazon SageMaker image classification algorithm now supports training in mixed-precision mode. This is controlled by the hyperparameter, precision_dtype, which can be set to ‘float32’ (default) or ‘float16’. In mixed-precision mode, the network computes the backward and forward pass in low-precision (float16) while maintaining the master weights in high-precision (float32). This enables the training to be faster while maintaining similar accuracy. By using the mixed-precision mode, the training time was reduced by 33 percent while obtaining the overall AUC of 0.821, which is similar to the one obtained with float32 training. The training time reduction was even greater when training using two instances for the high-resolution input (see the following section) and increased to 47 percent.

In [ ]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role,
                        train_instance_count=1,
                        train_instance_type='ml.p3.16xlarge',
                        train_volume_size = 50, train_max_run = 360000,
                        input_mode= 'File', output_path=s3_output_location,
                        sagemaker_session=sess)
# multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
#                                         image_shape = "3,224,224", num_classes=14,
#                                         mini_batch_size=256, 
#                                         resize=256,  epochs=100, 
#                                         learning_rate=0.0005, optimizer='adam', 
#                                         num_training_samples=80000,
#                                         augmentation_type = 'crop_color_transform',
#                                         precision_dtype='float32', multi_label = 1)
multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1,
                                        image_shape = "3,224,224", num_classes=14,
                                        mini_batch_size=256, resize=256, epochs=100, 
                                        learning_rate=0.0005, optimizer='adam', 
                                        num_training_samples=80000, use_weighted_loss=1, 
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float16', multi_label = 1)

train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)

### Training with high-resolution input

We then used the original input resolution by setting the image_shape parameter to 896×896. We used the use_weighted_loss feature and float32 precision for this training. We used this resolution because it allows the network to sample a 896×896 region from the 1024×1024 during data augmentation. Since the high resolution will use more memory, typically batch_size is reduced to train the network. However, because Amazon SageMaker image classification supports distributed training, we were able to maintain the batch_size by running the training across multiple instances. This is done by setting the [instance_count parameter](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTrainingJob.html) in the Amazon SageMaker training to 2. The average AUC for this resolution increased to 0.830, particularly for classes such as nodule, which can benefit from high-resolution input. When we trained with mixed_precision set to 1, the average AUC was 0.825. The training was done using the same code as before but setting the train_instance_count = 2, image_shape=”3,896,896” and not setting the resize parameter.



In [ ]:
multilabel_ic = sagemaker.estimator.Estimator(training_image, role, train_instance_count=2,
                                                train_instance_type='ml.p3.16xlarge',
                                                train_volume_size = 50, train_max_run = 360000,
                                                input_mode= 'File', output_path=s3_output_location,
                                                sagemaker_session=sess)

multilabel_ic.set_hyperparameters(num_layers=50, use_pretrained_model=1, 
                                        image_shape = "3,896,896", num_classes=14,
                                        mini_batch_size=64, epochs=100, 
                                        learning_rate=0.00025, optimizer='adam', 
                                        num_training_samples=80000, use_weighted_loss=1, 
                                        augmentation_type = 'crop_color_transform',
                                        precision_dtype='float32', multi_label = 1)

train_data = sagemaker.session.s3_input(s3train, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
validation_data = sagemaker.session.s3_input(s3validation, distribution='FullyReplicated',
                                                content_type='application/x-recordio',
                                                s3_data_type='S3Prefix')
data_channels = {'train': train_data, 'validation': validation_data}
multilabel_ic.fit(inputs=data_channels, logs=True)

| Label | 224×224 | 224×224 with class balancing | 224×224 with mixed precision | 896×896 |
|-----------| :--------------------------: | :--------------------------: | :-------: | ---------: |
|Atelectasis | 0.772 |0.802 | 0.799 | 0.800 |
|Cardiomegaly | 0.859 | 0.899 | 0.906 | 0.884 |
|Effusion | 0.830 | 0.873 | 0.873 | 0.873 |
|Infiltration | 0.626 | 0.693 | 0.691 | 0.698 |
|Mass | 0.791 | 0.839 | 0.834 | 0.821 |
|Nodule | 0.716 | 0.743 | 0.751 | 0.817 |
|Pneumonia | 0.645 | 0.710 | 0.713 | 0.739 |
|Pneumothorax | 0.778 | 0.836 | 0.862 | 0.878 |
|Consolidation | 0.695 | 0.791 | 0.789 | 0.785 |
|Edema | 0.799 | 0.849 | 0.863 | 0.879| 
|Emphysema | 0.850 | 0.889 | 0.909 | 0.933 |
|Fibrosis | 0.764 | 0.791 | 0.811 | 0.822 |
|Pleural Thickening | 0.726 | 0.758 | 0.761 | 0.785 |
|Hernia | 0.903 | 0.929 | 0.940 | 0.911 |
|Average AUC | 0.768 | 0.814 | 0.821 | 0.830 |


### Deploy the model

***

A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the topic mixture representing a given document.

Image-classification only supports encoded .jpg and .png image formats as inference input for now. The output is the probability values for all classes encoded in JSON format, or in JSON Lines format for batch transform.

This section involves several steps,

1. [Create Model](#CreateModel) - Create model for the training output
1. [Batch Transform](#BatchTransform) - Create a transform job to perform batch inference.
1. [Host the model for realtime inference](#HostTheModel) - Create an inference endpoint and perform realtime inference.

## Create Model

We now create a SageMaker Model from the training output. Using the model we can create a Batch Transform Job or an Endpoint Configuration.

In [ ]:
%%time
import boto3
from time import gmtime, strftime

sage = boto3.Session().client(service_name='sagemaker') 

# get the name of the training job completed below for this variable
job_name="image-classification-2018-11-03-07-10-36-441"
model_name = "x-ray-image-classification-model"
print(model_name)
info = sage.describe_training_job(TrainingJobName=job_name)
model_data = info['ModelArtifacts']['S3ModelArtifacts']
print(model_data)

hosting_image = get_image_uri(boto3.Session().region_name, 'image-classification')

primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': model_data,
}

create_model_response = sage.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

### Realtime inference

We now host the model with an endpoint and perform realtime inference.

This section involves several steps,
1. [Create endpoint configuration](#CreateEndpointConfiguration) - Create a configuration defining an endpoint.
1. [Create endpoint](#CreateEndpoint) - Use the configuration to create an inference endpoint.
1. [Perform inference](#PerformInference) - Perform inference on some input data using the endpoint.
1. [Clean up](#CleanUp) - Delete the endpoint and model

#### Create Endpoint Configuration
At launch, we will support configuring REST endpoints in hosting with multiple models, e.g. for A/B testing purposes. In order to support this, customers create an endpoint configuration, that describes the distribution of traffic across the models, whether split, shadowed, or sampled in some way.

In addition, the endpoint configuration describes the instance type required for model deployment, and at launch will describe the autoscaling configuration.

In [ ]:
import time
from time import gmtime, strftime

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = model_name + '-epc-' + timestamp
endpoint_config_response = sage.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':model_name,
        'VariantName':'AllTraffic'}])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

#### Create Endpoint
Lastly, the customer creates the endpoint that serves up the model, through specifying the name and configuration defined above. The end result is an endpoint that can be validated and incorporated into production applications. This takes 9-11 minutes to complete.

In [ ]:
%%time
import time

sage = boto3.client('sagemaker')
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_name = model_name + '-ep-' + timestamp
print('Endpoint name: {}'.format(endpoint_name))

endpoint_params = {
    'EndpointName': endpoint_name,
    'EndpointConfigName': endpoint_config_name,
}
endpoint_response = sage.create_endpoint(**endpoint_params)
print('EndpointArn = {}'.format(endpoint_response['EndpointArn']))

In [ ]:
# get the status of the endpoint
response = sage.describe_endpoint(EndpointName=endpoint_name)
status = response['EndpointStatus']
print('EndpointStatus = {}'.format(status))


# wait until the status has changed
sage.get_waiter('endpoint_in_service').wait(EndpointName=endpoint_name)


# print the status of the endpoint
endpoint_response = sage.describe_endpoint(EndpointName=endpoint_name)
status = endpoint_response['EndpointStatus']
print('Endpoint creation ended with EndpointStatus = {}'.format(status))

if status != 'InService':
    raise Exception('Endpoint creation failed.')

If you see the message,

> `Endpoint creation ended with EndpointStatus = InService`

then congratulations! You now have a functioning inference endpoint. You can confirm the endpoint configuration and status by navigating to the "Endpoints" tab in the AWS SageMaker console.

We will finally create a runtime object from which we can invoke the endpoint.

#### Perform Inference
Finally, the customer can now validate the model for use. They can obtain the endpoint from the client library using the result from previous operations, and generate classifications from the trained model using that endpoint.

In [ ]:
import boto3
runtime = boto3.Session().client(service_name='runtime.sagemaker') 

In [ ]:
!head -10 chestxraytest.lst

In [ ]:
file_name = '/home/ec2-user/SageMaker/images/00000003_002.png'
# test image
from IPython.display import Image
Image(file_name)  

In [ ]:
import json
import numpy as np
with open(file_name, 'rb') as f:
    payload = f.read()
    payload = bytearray(payload)
endpoint_name = 'x-ray-image-classification-model-ep--2018-11-03-16-26-35'
response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='application/x-image', 
                                   Body=payload)
result = response['Body'].read()
# result will be in json format and convert it to ndarray
result = json.loads(result)
print(result)
# the result will output the probabilities for all classes
# find the class with maximum probability and print the class index
index = np.argmax(result)
print(index)
disease_list = ['Atelectasis', 'Consolidation', 'Infiltration', 'Pneumothorax', 'Edema', 'Emphysema', \
                   'Fibrosis', 'Effusion', 'Pneumonia', 'Pleural_Thickening', 'Cardiomegaly', 'Nodule', 'Mass', \
                   'Hernia']


print("Top Result: label - " + disease_list[index] + ", probability - " + str(result[index]))
for idx, val in enumerate(result):
    print('%s:\t%f \n'%(disease_list[idx], result[idx]), end='')